In [38]:
from binance.client import Client
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import tensorflow as tf


In [37]:
api_key = 'iyJXPaZztWrimkH6V57RGvStFgYQWRaaMdaYBQHHIEv0mMY1huCmrzTbXkaBjLFh'
api_secret = 'hmrus7zI9PW2EXqsDVovoS2cEFRVsxeETGgBf4XJInOLFcmIXKNL23alGRNRbXKI'
client = Client(api_key, api_secret)

In [10]:
symbol = 'ETHUSDT'
interval = '3m'

In [11]:
klines = client.get_klines(symbol=symbol, interval=interval)


In [12]:
data = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])


In [14]:
data['timestamp'] = pd.to_datetime(data['timestamp'], unit='ms')
data

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2023-12-10 04:27:00,2352.20000000,2352.92000000,2352.19000000,2352.91000000,158.01040000,1702182599999,371702.38934400,512,116.30640000,273598.24056000,0
1,2023-12-10 04:30:00,2352.91000000,2352.98000000,2352.69000000,2352.69000000,179.72040000,1702182779999,422858.22051000,585,78.15800000,183889.74183000,0
2,2023-12-10 04:33:00,2352.70000000,2353.56000000,2352.69000000,2353.14000000,241.26060000,1702182959999,567745.17458400,812,192.89260000,453921.70983500,0
3,2023-12-10 04:36:00,2353.13000000,2354.86000000,2353.13000000,2354.43000000,570.18700000,1702183139999,1342310.33783300,1228,350.90980000,826024.60992600,0
4,2023-12-10 04:39:00,2354.44000000,2354.79000000,2354.17000000,2354.79000000,184.50750000,1702183319999,434434.72925800,648,127.64170000,300539.23186400,0
...,...,...,...,...,...,...,...,...,...,...,...,...
495,2023-12-11 05:12:00,2245.69000000,2245.69000000,2243.65000000,2244.91000000,308.87540000,1702271699999,693378.81820500,941,129.06430000,289706.75642500,0
496,2023-12-11 05:15:00,2244.91000000,2245.50000000,2243.52000000,2244.27000000,401.02990000,1702271879999,900233.61345400,1051,191.96230000,430873.92992100,0
497,2023-12-11 05:18:00,2244.27000000,2244.81000000,2239.86000000,2239.87000000,595.10110000,1702272059999,1334365.08126800,1372,225.65270000,506021.21814100,0
498,2023-12-11 05:21:00,2239.87000000,2240.86000000,2236.72000000,2237.40000000,406.36500000,1702272239999,909926.75856700,1357,172.57850000,386432.78093600,0


In [15]:
def calculate_sma(price_history, window_size):
    if len(price_history) < window_size:
        return None
    else:
        return sum(price_history[-window_size:]) / window_size

In [42]:
window_size = 10  # Adjust the window size as needed
data['sma'] = data['close'].rolling(window=window_size).mean()


In [43]:
print(data)


              timestamp           open           high            low  \
9   2023-12-10 04:54:00  2354.24000000  2354.50000000  2353.96000000   
10  2023-12-10 04:57:00  2354.50000000  2354.62000000  2354.49000000   
11  2023-12-10 05:00:00  2354.62000000  2355.02000000  2353.87000000   
12  2023-12-10 05:03:00  2353.87000000  2353.88000000  2352.19000000   
13  2023-12-10 05:06:00  2352.19000000  2352.72000000  2352.00000000   
..                  ...            ...            ...            ...   
494 2023-12-11 05:09:00  2244.80000000  2245.89000000  2244.12000000   
495 2023-12-11 05:12:00  2245.69000000  2245.69000000  2243.65000000   
496 2023-12-11 05:15:00  2244.91000000  2245.50000000  2243.52000000   
497 2023-12-11 05:18:00  2244.27000000  2244.81000000  2239.86000000   
498 2023-12-11 05:21:00  2239.87000000  2240.86000000  2236.72000000   

             close        volume     close_time quote_asset_volume  \
9    2354.50000000  107.23240000  1702184219999    252456.4438760

In [23]:
data['target'] = data['close'].shift(-1)

data = data.dropna()


C:\Users\admin\AppData\Local\Temp\ipykernel_11388\382399669.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'] = data['close'].shift(-1)


In [25]:
X = data[['close', 'sma']].values
y = data['target'].values

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [56]:
X_test

array([[2243.57 , 2238.929],
       [2344.76 , 2343.302],
       [2237.4  , 2236.807],
       [2236.77 , 2243.325],
       [2230.61 , 2290.206],
       [2358.79 , 2357.119],
       [2353.48 , 2354.973],
       [2351.67 , 2352.496],
       [2243.63 , 2238.634],
       [2349.82 , 2352.32 ],
       [2353.72 , 2353.234],
       [2336.33 , 2334.71 ],
       [2326.6  , 2331.776],
       [2348.36 , 2347.39 ],
       [2362.93 , 2364.616],
       [2346.77 , 2351.951],
       [2229.14 , 2269.962],
       [2240.59 , 2260.84 ],
       [2357.83 , 2362.693],
       [2352.11 , 2349.099],
       [2342.77 , 2344.178],
       [2345.3  , 2345.125],
       [2315.   , 2328.632],
       [2346.55 , 2347.523],
       [2341.22 , 2346.202],
       [2342.92 , 2345.242],
       [2344.44 , 2342.436],
       [2349.22 , 2345.636],
       [2357.22 , 2350.627],
       [2352.39 , 2350.365],
       [2322.62 , 2332.505],
       [2337.03 , 2335.487],
       [2363.19 , 2358.403],
       [2371.19 , 2369.987],
       [2350.4

In [39]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=(X.shape[1],))
])

In [40]:
model.compile(optimizer='sgd', loss='mean_squared_error')


In [52]:
X_test = X_test.astype(np.float32)
y_test = y_test.astype(np.float32)

In [ ]:
X_train = X_train.astype(np.float32)
y_train = y_train.astype(np.float32)

In [57]:
num_epochs = 10  # Adjust the number of epochs as needed
history = model.fit(X_train, y_train, epochs=num_epochs, validation_split=0.2, verbose=1)
history

Epoch 1/10
10/10 [==============================] - 0s 6ms/step - loss: nan - val_loss: nan
Epoch 2/10
10/10 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 3/10
10/10 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 4/10
10/10 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 5/10
10/10 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 6/10
10/10 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 7/10
10/10 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan
Epoch 8/10
10/10 [==============================] - 0s 3ms/step - loss: nan - val_loss: nan
Epoch 9/10
10/10 [==============================] - 0s 2ms/step - loss: nan - val_loss: nan
Epoch 10/10
10/10 [==============================] - 0s 4ms/step - loss: nan - val_loss: nan


In [53]:
y_pred = model.predict(X_test).flatten()


4/4 [==============================] - 0s 2ms/step


In [54]:
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

ValueError: Input contains NaN.